In [ ]:
import pickle
import os

import AE as ae
import numpy as np

import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

## Load Data

In [ ]:
directory = 'INSERT DIRECTORY' # Directory where baseline generated by Simulation.py is saved.
    
background_file = '0_SNR_data.pkl' # Baseline eventless data filename.

thresholds = [0, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 0.9999, 1]

# Load Background data
with open(os.path.join(directory, background_file), 'rb') as f:
    background_data = pickle.load(f)[0]

In [ ]:
### Constants

window_size = 100 # Must match network input size
step_size = 10

model_dir = 'INSERT FOLDER NAME' # Path to network model save location
is_training = False

## Analyse Background

In [ ]:
####### Normalise Data Here #########
background_mean = background_data.mean()
background_std = background_data.std()

background_data_norm = (background_data - background_mean) / background_std

In [ ]:
if is_training:
    # Generate and Train AE model
    model, history = ae.train_model(background_data_norm, window_size, step_size)
    
    model.save(model_dir)
    with open('{}_hist.pkl'.format(model_dir), 'wb') as f:
        pickle.dump(history.history, f)
else:
    # Load pretrained AE model
    model = tf.keras.models.load_model(model_dir)
    model_out_dir = '/mnt/albrecht-neuralnet/CDW/Thesis Work by Chapter/Anomaly Detection/AE/AE_100_1 Outputs'
    with open(os.path.join(model_out_dir, '{}_hist.pkl'.format(model_dir)), 'rb') as f:
        history = pickle.load(f) 
    
model.summary()

In [ ]:
# Calculate the Gaussian Distribution of Background MAE values.
background_results = ae.analyse_background(background_data_norm, model, window_size, step_size)

mae_losses, bin_edges, bin_counts, gauss_params, norm_factor = background_results

In [ ]:
# Save the generated background MAEs
with open('INSERT FILENAME', 'wb') as f:
    pickle.dump(mae_losses, f)

## Generate Thresholds

In [ ]:
### Run if background analysis fails to converge ###
# Manual analysis for normalisation
from scipy.integrate import quad
area = quad(ae.gauss, 0.1, 0.2, args=tuple(gauss_params), points=(0.1, 0.2))

norm_factor= area[0]
norm_factor

In [ ]:
bin_mids = bin_edges[:-1] + (np.diff(bin_edges) / 2)
gauss_params_norm = gauss_params.copy()
gauss_params_norm[0] /= norm_factor


fig, ax = plt.subplots()
ax.bar(bin_mids, bin_counts / norm_factor, width=np.diff(bin_edges))
ax.plot(bin_mids, ae.gauss(bin_mids, *gauss_params_norm), color='red')

In [ ]:
# Calculate the Threshold Values from the Distribution.
mae_thresholds = []
mae_losses = np.array(mae_losses)
for thresh in thresholds:
    print(thresh)
    val = ae.limit_finder(gauss_params_norm, mae_losses.min(), mae_losses.max(), thresh)
    mae_thresholds.append(val)
mae_thresholds

In [ ]:
thresh_dict = dict(zip(thresholds, mae_thresholds))
thresh_dict

In [ ]:
# Save generated thresholds
with open('INSERT FILENAME', 'wb') as f:
    pickle.dump(thresh_dict, f)

In [ ]:
# Save background analysis results
with open('INSERT FILENAME', 'wb') as f:
    pickle.dump((background_mean, background_std), f)

## Figures

In [ ]:
# Plot the background distribution of MAEs
fig, ax = plt.subplots(figsize=(6, 4), dpi=600)
ax.bar(bin_mids, bin_counts / norm_factor, width=np.diff(bin_edges))
ax.plot(bin_mids, ae.gauss(bin_mids, *gauss_params_norm), color='red')

ax.set_xlabel("MAE", weight='bold')
ax.set_ylabel("Nomralised Counts", weight='bold')

ax.tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    ax.spines[axis].set_linewidth(1.5)

text_str = "Gaussian\nMean: {:.2f}\nStd: {:.2f}".format(gauss_params[1], gauss_params[2])
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
ax.text(0.05, 0.95, text_str, transform=ax.transAxes,
        verticalalignment='top', bbox=props)

# fig.savefig('AE Background.png')